In [37]:
import pandas as pd
import numpy as np

<h2> Remedy for NaN industry code </h2>

In [46]:
df = pd.read_csv("metadata_reports_noduplicates_with_industry.csv")

In [143]:
df_industry_mapping = pd.read_csv("Ticker_ConpanyName_Industry.csv")
df_industry_mapping = df_industry_mapping.dropna(subset=['ggroup', "conm", "TICKER"]).reset_index(drop=True)

In [134]:
companies_ref = df_industry_mapping.conm.str.upper().str.strip().unique()

In [135]:
ticker_ref = df_industry_mapping.TICKER.str.upper().str.strip().unique()

In [136]:
no_industry_index = df['industry'].index[df['industry'].apply(np.isnan)].tolist()

In [137]:
df_no_industry = df.iloc[df['industry'].index[df['industry'].apply(np.isnan)],:]

In [170]:
fixed_index = []
not_fixed_index = []
for i in no_industry_index:
    row = df.iloc[i, :]
    company_name = str(row["Company Name"]).upper().strip()
    ticker = str(row["TICKER"]).strip()
    if ticker == "BRKa":
        ticker = "BRK"
    if company_name in companies_ref:
        latest_year = df_industry_mapping[df_industry_mapping.conm == company_name]["year"].max()
        row_ref = df_industry_mapping[(df_industry_mapping.conm == company_name) & (df_industry_mapping.year == latest_year)].max()
        df.iloc[i, -3] = int(row_ref["ggroup"])
        df.iloc[i, -4] = int(row_ref["gsector"])
        df.iloc[i, -5] = int(row_ref["PERMNO"])
        fixed_index.append(i)
    elif ticker in ticker_ref:
        latest_year = df_industry_mapping[df_industry_mapping.TICKER == ticker]["year"].max()
        row_ref = df_industry_mapping[(df_industry_mapping.TICKER == ticker) & (df_industry_mapping.year == latest_year)].max()
        df.iloc[i, -3] = int(row_ref["ggroup"])
        df.iloc[i, -4] = int(row_ref["gsector"])
        df.iloc[i, -5] = int(row_ref["PERMNO"])
        fixed_index.append(i)
    else:
        not_fixed_index.append(i)

In [183]:
len(fixed_index)

2419

In [198]:
Russell_ticker_list = pd.read_excel("./list_SP500.xlsx", sheet_name="Russell 3000")[".RUA"].str.split('.').str[0].tolist()
SP500_ticker_list = pd.read_excel("./list_SP500.xlsx", sheet_name="S&P500")[".SP500"].str.split('.').str[0].tolist()

In [199]:
not_in_russell = []
in_russell = []
not_in_SP500 = []
in_SP500 = []
for ticker in df.iloc[not_fixed_index].TICKER.unique():
    if ticker not in Russell_ticker_list:
        not_in_russell.append(ticker)
    else:
        in_russell.append(ticker)
    if ticker not in SP500_ticker_list:
        not_in_SP500.append(ticker)
    else:
        in_SP500.append(ticker)

In [200]:
in_SP500

['ACN',
 'AIV',
 'ALLE',
 'AMCR',
 'AMT',
 'AON',
 'APTV',
 'ARE',
 'AVB',
 'BFb',
 'BXP',
 'CB',
 'CCI',
 'CCL',
 'DLR',
 'DRE',
 'EQIX',
 'EQR',
 'ESS',
 'ETN',
 'EXR',
 'FRT',
 'FTI',
 'GRMN',
 'HST',
 'INFO',
 'IRM',
 'IVZ',
 'JCI',
 'KIM',
 'LIN',
 'LYB',
 'MAA',
 'MDT',
 'MYL',
 'NCLH',
 'NLSN',
 'O',
 'PEAK',
 'PLD',
 'PNR',
 'PRGO',
 'PSA',
 'RCL',
 'RE',
 'REG',
 'SBAC',
 'SLB',
 'SLG',
 'SPG',
 'STE',
 'STX',
 'TEL',
 'TT',
 'UDR',
 'VNO',
 'VTR',
 'WELL',
 'WLTW',
 'WY']

In [215]:
not_in_russell

['AMCR', 'EI', 'FTI', 'HPE (1)', 'STX', 'TEL']

In [216]:
not_in_SP500

['EI', 'HPE (1)']

In [182]:
df.iloc[not_fixed_index].groupby(["TICKER"]).count().reset_index().sort_values(by=['Available'], ascending=False)

,TICKER,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,...,SecondaryTicker,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup,industry,quarter_year
23,FTI,366,366,366,366,355,114,343,366,364,...,366,366,352,366,366,0,0,0,0,366
0,ACN,299,299,299,299,297,90,293,299,299,...,299,299,276,299,299,0,0,0,0,299
35,MDT,294,294,294,294,291,44,286,294,294,...,294,294,290,294,294,0,0,0,0,294
16,EI,291,291,291,291,288,32,284,291,291,...,291,291,278,291,291,0,0,0,0,291
49,SLB,250,250,250,250,250,17,247,250,250,...,250,250,231,250,250,0,0,0,0,250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,NLSN,82,82,82,82,81,7,82,82,82,...,82,82,77,82,82,0,0,0,0,82
34,MAA,79,79,79,79,79,0,79,79,79,...,79,79,76,79,79,0,0,0,0,79
52,STE,67,67,67,67,67,3,64,67,67,...,67,67,66,67,67,0,0,0,0,67
3,AMCR,46,46,46,46,44,10,45,46,46,...,46,46,45,46,46,0,0,0,0,46


<h2> Dropping rows with ticker company mismatch <h2>

In [208]:
df['industry'] = df['ggroup']
df['quarter_year'] = df["year"].astype("str") + " q" + df["quarter"].astype("str")
df2= df.dropna(subset=['industry']).reset_index(drop=True)
df2['industry-quarter'] = list(zip(df2.industry, df2.quarter_year))
list_industries_quarter = df2.groupby('industry-quarter').count().reset_index()['industry-quarter'].tolist()

In [210]:
temp=df2.groupby(['TICKER','Company Name']).count()['Available'].reset_index()
temp2=temp.groupby(['TICKER']).apply(lambda x: x.sort_values(['Available'], ascending=False))
del temp2['TICKER']
temp2=temp2.reset_index()
temp3=temp2.groupby('TICKER').first().reset_index()

In [211]:
index_no_issue_list = []
for i in range(df2.shape[0]):
    row = df2.iloc[i, :]
    ticker = str(row["TICKER"]).split(".")[0]
    company_name = str(row["Company Name"])
    if company_name == max(temp3[temp3["TICKER"] == ticker]["Company Name"]):
        index_no_issue_list.append(i)

In [212]:
df_no_issue = df2.iloc[index_no_issue_list]

In [213]:
df_no_issue

,Available,Date,Doc Grp,Info,Prim. Ticker,Second. Ticker,RI,Title,Pages,Price,...,Match_Ticker,GenderAnalyst,year,quarter,PERMNO,gsector,ggroup,industry,quarter_year,industry-quarter
0,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Increase, TargetPrice Increase","3Q18 Recap: Maintaining Our Rating, Raising Ou...",9.0,92.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
1,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,Morningstar | Raising Our FVE After American S...,13.0,69.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
2,2018-11-25,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, TargetPrice Increase",Morningstar | Rising Fuel Costs Raise American...,25.0,195.5,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
3,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,Estimate Increase,3Q18 Recap; Strategic Initiatives Aimed At Dri...,5.0,80.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
4,2018-11-02,2018-10-26,Inv.,pdf,AAL.OQ,NaN,"Estimate Revision, Recommendation Decrease, Ta...",American Airlines Group Inc.,9.0,69.0,...,1,male,2018,4,21020.0,20.0,2030.0,2030.0,2018 q4,"(2030.0, 2018 q4)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89296,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation - 4Q Core EPS $0.80 - So...,12.0,92.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"
89297,2018-01-29,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Solid Fees Offset Slightly Smaller Balance She...,7.0,100.0,...,1,NaN,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"
89298,2018-02-05,2018-01-22,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,4Q17 - Post Conference Call Model Update,7.0,69.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"
89299,2018-01-17,2018-01-10,Inv.,pdf,"ZIONP.OQ, ZION.OQ",NaN,NaN,Zions Bancorporation,9.0,69.0,...,1,male,2018,1,84129.0,40.0,4010.0,4010.0,2018 q1,"(4010.0, 2018 q1)"


In [214]:
df.to_csv("cleaned_metadata_reports_noduplicates_with_industry.csv")

In [84]:
# fixed_list = []
# for i in index_issue_list:
#     row = df2.iloc[i, :]
#     if not row["Prim. Ticker"] is np.nan:
#         primary_tickers = row["Prim. Ticker"].split(",")
#     else:
#         continue
#     found = False
#     for p_ticker in primary_tickers:
#         if p_ticker.split(".")[0] == str(row["TICKER"]):
#             found = True
#             break
#     if found:
#         fixed_list.append(i)